In [1]:
from  correctingagent.pddl import problem_def
from correctingagent.util.colour_dict import colour_dict, simple_colour_dict
from correctingagent.world import world
import os
import random

In [2]:
datapath = '/home/mappelgren/Desktop/correcting-agent/data'
dataset_name = 'bigthreerules'
dataset_dir = os.path.join(datapath, dataset_name)
for d in os.listdir(dataset_dir):
    name = d.replace('dataset_name', dataset_name)
    old = os.path.join(dataset_dir, d)
    new = os.path.join(dataset_dir, name)
    os.rename(old, new)

In [11]:
def sample_colour(colour_dict):
    category = random.choice(list(colour_dict.keys()))
    return random.choice(colour_dict[category])

def sample_colours(n, colour_dict=colour_dict):
    return [sample_colour(colour_dict) for i in range(n)]
 
def sample_from_category(category):
    try:
        choices = colour_dict[category]
        return random.choice(choices)
    except KeyError:
        return category

In [4]:
def generate_dataset(N, rules, directory, colour_dict=colour_dict):
    os.makedirs(directory, exist_ok=True)
    dataset = []
    while len(dataset) < N:
        colours = sample_colours(10, colour_dict=colour_dict)
        with open('tmp/generation.pddl', 'w') as f:
            f.write(problem_def.create_problem(colours, rules).asPDDL())
        w = world.PDDLWorld('blocks-domain.pddl', 'tmp/generation.pddl')
        if not w.test_failure():
            file_name = '{}/problem{}.pddl'.format(directory,len(dataset)+1)
            os.rename('tmp/generation.pddl', file_name)
            dataset.append(file_name)

In [10]:
random.random()

0.5657057491819072

In [112]:
class Constraint(object):
    
    def __init__(self, rule):
        c1 = rule.first_obj[0]
        c2 = rule.second_obj[0]
        
        
        if rule.constrained_obj == 'first':
            self.str = "{} <= {}".format(c1, c2)
            self.smaller = c1
            self.bigger = c2
        else:
            self.str = "{} <= {}".format(c2, c1)
            self.smaller = c2
            self.bigger = c1
    
    def test_constraint(self, colour_counts):
        return colour_counts[self.smaller] <= colour_counts[self.bigger]
    

def generate_colour_counts(colour_dict=colour_dict):
    colour_counts = {category:0 for category in colour_dict.keys()}
    for i in range(10):
        colour_counts[random.choice(list(colour_counts.keys()))] += 1
    return colour_counts

def generate_biased_sample(rules, colour_dict=colour_dict):
    important_colours = set()
    colour_count = {category:0 for category in colour_dict.keys()}
    for rule in rules:
        c1 = rule.first_obj[0]
        c2 = rule.second_obj[0]
        important_colours.add(c1)
        important_colours.add(c2)
    important_colours = list(important_colours)
    
    
    r = random.random()
    if r < 0.2:
        c = random.choice(list(colour_dict.keys()))
        return c
    else:
        return random.choice(important_colours)
            
def generate_full_sample(N, rules):
    colour_count = {category:0 for category in colour_dict.keys()}
    for i in range(N):
        colour_count[generate_biased_sample(rules)] += 1
    return colour_count
        
def generate_biased_colour_counts(rules, colour_dict=colour_dict):
    constraints = [Constraint(rule) for rule in rules]
    colour_count = generate_full_sample(10, rules)
    while not verify_colour_counts(rules, colour_count):
        colour_count = generate_full_sample(10, rules)
    return colour_count
    
    
# def generate_biased_colour_counts(rules, colour_dict=colour_dict):
#     constraints = [Constraint(rule) for rule in rules]
#     colour_count = {category:0 for category in colour_dict.keys()}
#     num_colours = 0
#     while num_colours < 10
#     r = random.random()
#     if r < 0.2:
#         c = random.choice(list(colour_dict.keys()))
#         colour_count[c] += 1
#         if not all([constraint.test_constraint(colour_count) for constraint in constraints]):
#             colour_count[c] -= 1
#     else:
#         rule = random.choice(rules)
#         c1 = rule.first_obj[0]
#         c2 = rule.second_obj[0]
#         colour_count[c1] += 1
#         colour_count[c2] += 1
#         if not all([constraint.test_constraint(colour_count) for constraint in constraints]):
#             colour_count[c1] -= 1
#             colour_count[c2] -= 1
#             if rule.constrained_obj == 'first':
#                 colour_count[]
            
#         colours.append(sample_from_category(c1))
#         colours.append(sample_from_category(c2))

def verify_colour_counts(rules, colour_counts):
    constraints = [Constraint(rule) for rule in rules]

    return all([constraint.test_constraint(colour_counts) for constraint in constraints])

def generate_from_colour_count(rules, colour_count):
    colours = []
    for colour, count in colour_count.items():
        for i in range(count):
            colours.append(sample_from_category(colour))
    random.shuffle(colours)
    return colours

In [113]:
rules = [rule1, rule3]
cc = generate_biased_colour_counts(rules)
print(cc)
generate_from_colour_count(rules, cc)


{'red': 4, 'green': 1, 'blue': 4, 'yellow': 0, 'purple': 0, 'pink': 1, 'orange': 0}


['crimson',
 'royalblue',
 'red',
 'royalblue',
 'darkred',
 'green',
 'dodgerblue',
 'navy',
 'crimson',
 'magenta']

In [114]:
def generate_biased_dataset(N, rules, directory, colour_dict=colour_dict):
    
    os.makedirs(directory, exist_ok=True)
    dataset = []
    while len(dataset) < N:
        colours = []
        
        cc = generate_biased_colour_counts(rules)
        colours = generate_from_colour_count(rules, cc)
        
        with open('../data/tmp/generation.pddl', 'w') as f:
            f.write(problem_def.create_problem(colours, rules).asPDDL())
        w = world.PDDLWorld('blocks-domain.pddl', '../data/tmp/generation.pddl')
        if not w.test_failure():
            file_name = '{}/problem{}.pddl'.format(directory,len(dataset)+1)
            os.rename('../data/tmp/generation.pddl', file_name)
            dataset.append(file_name)
        
        

In [116]:
def generate_dataset_set(N_datasets, N_data, rules, dataset_name, colour_dict=colour_dict):
    data_path = '/home/mappelgren/Desktop/correcting-agent/data'
    top_path = os.path.join(data_path, dataset_name)
    for i in range(N_datasets):
        dataset_path = os.path.join(top_path, 'dataset_name{}'.format(i))
        os.makedirs(dataset_path, exist_ok=True)
        
        generate_biased_dataset(N_data, rules, dataset_path, colour_dict=colour_dict)
    

In [119]:
generate_dataset_set(50, 50, [rule1, rule3, rule9], 'bigthreerules')

Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-

Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-

Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-

Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-

Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-

...

Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-

Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/

Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-

Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-

Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/

In [115]:
generate_biased_dataset(10, [rule1, rule6, rule8], 'test')

Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem ../data/tmp/generation.pddl
Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-

In [15]:
rule1 = problem_def.Ruledef(['red'], ['blue'], 'first')
rule2 = problem_def.Ruledef(['green'], ['yellow'], 'second')
rule3 = problem_def.Ruledef(['red'], ['blue'], 'second')
rule4 = problem_def.Ruledef(['green'], ['maroon'], 'first')
rule5 = problem_def.Ruledef(['purple'], ['orange'], 'first')
rule6 = problem_def.Ruledef(['pink'], ['red'], 'first')
rule7 = problem_def.Ruledef(['pink'], ['red'], 'second')
rule8 = problem_def.Ruledef(['purple'], ['red'], 'first')
rule9 = problem_def.Ruledef(['purple'], ['orange'], 'second')

In [6]:
generate_dataset(50, [rule1, rule9], 'tworules2')

Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain block

In [6]:
generate_dataset(50, [rule1, rule2, rule5], 'threerules')

Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain block

Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain block

Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/

In [6]:
generate_dataset(20, [rule1], 'oneruletest')
generate_dataset(20, [rule1], 'simplecolourstest', colour_dict=simple_colour_dict)
generate_dataset(20, [rule1, rule2], 'tworulestest')
generate_dataset(20, [rule1, rule4], 'maroontest')
generate_dataset(20, [rule1, rule3], 'bijectiontest')

Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain block

Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain block

Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/

In [10]:
generate_dataset(50, [rule1, rule6, rule8], 'triplicatered')
#generate_dataset(50, [rule1, rule7], 'duplicatered')

Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain block

Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain block

Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/

Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain block

In [12]:
generate_dataset(20, [rule1, rule6, rule8], 'triplicateredtest')
#generate_dataset(20, [rule1, rule7], 'duplicateredtest')

Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain block

Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain blocks-domain.pddl
Parsing problem tmp/generation.pddl
Parsing domain block